In [1]:
import pandas as pd
import numpy as np
import lightgbm
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
df_train = pd.read_csv('F:\\career\\Udemy_projects\\HR analytics\\train_LZdllcl.csv')
df_test = pd.read_csv('F:\\career\\Udemy_projects\\HR analytics\\test_2umaH9m.csv')
df_test['is_promoted'] = 0

In [3]:
df_test.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77,0
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51,0
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47,0
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65,0
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61,0


In [4]:
df_merged = pd.concat([df_train,df_test], axis=0)

In [5]:
print('Train Length: ',len(df_train))
print('Test Length: ',len(df_test))
print('Total Length: ',len(df_merged))

Train Length:  54808
Test Length:  23490
Total Length:  78298


In [6]:
# setting empid as index
df_merged.set_index('employee_id', inplace=True)

#### Now applying feature transformation to the whole dataset


In [7]:
df_merged.columns

Index(['department', 'region', 'education', 'gender', 'recruitment_channel',
       'no_of_trainings', 'age', 'previous_year_rating', 'length_of_service',
       'KPIs_met >80%', 'awards_won?', 'avg_training_score', 'is_promoted'],
      dtype='object')

In [8]:
#df_merged['kpi_prev_rating'] = df_merged['KPIs_met >80%'] + df_merged['previous_year_rating']

In [9]:

# fill education missing values with bachelors'
df_merged['education'].value_counts()
df_merged['education'].fillna("Bachelor's", inplace = True)
df_merged.education.isnull().sum()


0

In [10]:
# Filling missing values: we have filled previous_yr_rating missing values with 3.9 if is_promoted = 1 else 3.26

df_merged['previous_year_rating'] = df_merged.apply(
    lambda row: 3.267910689246036 if np.isnan(row['previous_year_rating']) and row['is_promoted']==0 
    else (3.986139986139986 if np.isnan(row['previous_year_rating']) and row['is_promoted']==1 else row['previous_year_rating'])
    , axis=1)

In [11]:
print(df_merged.previous_year_rating.isnull().sum())
print(np.mean(df_merged.previous_year_rating))
print(df_merged.previous_year_rating.max())
print(df_merged.previous_year_rating.min())

0
3.3304528542620693
5.0
1.0


In [12]:
# renaming some columns

df_merged = df_merged.rename(columns = {"KPIs_met >80%":"KPIs_met_80", "awards_won?":"awards_won"}) 

In [13]:
df_merged.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met_80,awards_won,avg_training_score,is_promoted
employee_id,,,,,,,,,,,,,
65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [14]:
df_merged['kpi_prev_rating'] = df_merged['KPIs_met_80'] + df_merged['previous_year_rating']

In [15]:
df_merged['year_tobe_served'] = 60 - df_merged['age'] + df_merged['length_of_service']

In [16]:
df_merged = df_merged.drop(columns=['length_of_service','KPIs_met_80','previous_year_rating','age'])

In [17]:
# Do the one hot coding as required
#emp_id = df['employee_id']
#df = df.drop(columns=['employee_id'])
df_merged = pd.get_dummies(df_merged, drop_first=True)
df_merged.head()

,no_of_trainings,awards_won,avg_training_score,is_promoted,kpi_prev_rating,year_tobe_served,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,region_region_10,region_region_11,region_region_12,region_region_13,region_region_14,region_region_15,region_region_16,region_region_17,region_region_18,region_region_19,region_region_2,region_region_20,region_region_21,region_region_22,region_region_23,region_region_24,region_region_25,region_region_26,region_region_27,region_region_28,region_region_29,region_region_3,region_region_30,region_region_31,region_region_32,region_region_33,region_region_34,region_region_4,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,education_Below Secondary,education_Master's & above,gender_m,recruitment_channel_referred,recruitment_channel_sourcing
employee_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
65438,1,0,49,0,6.0,33,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
65141,1,0,60,0,5.0,34,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7513,1,0,50,0,3.0,33,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2542,2,0,50,0,1.0,31,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
48945,1,0,73,0,3.0,17,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [18]:
df_merged.columns

Index(['no_of_trainings', 'awards_won', 'avg_training_score', 'is_promoted',
       'kpi_prev_rating', 'year_tobe_served', 'department_Finance',
       'department_HR', 'department_Legal', 'department_Operations',
       'department_Procurement', 'department_R&D',
       'department_Sales & Marketing', 'department_Technology',
       'region_region_10', 'region_region_11', 'region_region_12',
       'region_region_13', 'region_region_14', 'region_region_15',
       'region_region_16', 'region_region_17', 'region_region_18',
       'region_region_19', 'region_region_2', 'region_region_20',
       'region_region_21', 'region_region_22', 'region_region_23',
       'region_region_24', 'region_region_25', 'region_region_26',
       'region_region_27', 'region_region_28', 'region_region_29',
       'region_region_3', 'region_region_30', 'region_region_31',
       'region_region_32', 'region_region_33', 'region_region_34',
       'region_region_4', 'region_region_5', 'region_region_6',
      

In [19]:
X = df_merged[:54808]
X_val = df_merged[54808:]

In [20]:
X_val.head()

,no_of_trainings,awards_won,avg_training_score,is_promoted,kpi_prev_rating,year_tobe_served,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,region_region_10,region_region_11,region_region_12,region_region_13,region_region_14,region_region_15,region_region_16,region_region_17,region_region_18,region_region_19,region_region_2,region_region_20,region_region_21,region_region_22,region_region_23,region_region_24,region_region_25,region_region_26,region_region_27,region_region_28,region_region_29,region_region_3,region_region_30,region_region_31,region_region_32,region_region_33,region_region_34,region_region_4,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,education_Below Secondary,education_Master's & above,gender_m,recruitment_channel_referred,recruitment_channel_sourcing
employee_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8724,1,0,77,0,4.267911,37,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
74430,1,0,51,0,3.000000,34,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
72255,1,0,47,0,1.000000,33,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
38562,3,0,65,0,2.000000,38,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
64486,1,0,61,0,4.000000,37,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [21]:
Y = X['is_promoted']
X = X.drop(columns=['is_promoted'])

In [22]:
len(X_val)

23490

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=0)

In [24]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train2 = pd.DataFrame(sc_X.fit_transform(X_train))
X_test2 = pd.DataFrame(sc_X.fit_transform(X_test))


X_train2.columns = X_train.columns.values
X_test2.columns = X_test.columns.values

X_train2.index = X_train.index.values
X_test2.index = X_test.index.values

In [27]:
# Light GBM
categorical_features = [c for c, col in enumerate(X.columns) if 'cat' in col]
train_data = lightgbm.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(X_test, label=y_test)


parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'f1',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}


lgb_clf = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=100)
y_pred = lgb_clf.predict(X_test)




ValueError: For early stopping, at least one dataset and eval metric is required for evaluation

In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
#print(confusion_matrix(y_test, y_pred, labels=['True', 'Is_Promoted']))

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [201]:
from xgboost import XGBClassifier
xgb_classifier = XGBClassifier(base_score=0.5,n_estimators=250, max_depth=6,learning_rate= 0.25, objective= 'binary:logistic')
xgb_classifier.fit(X_train,y_train)
y_pred = xgb_classifier.predict(X_test)

In [204]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels=['True', 'Is_Promoted']))

0.9486407589855865
0.4201954397394137
0.5789080029917726
0.9302884615384616


ValueError: At least one label specified must be in y_true

In [187]:
from sklearn.metrics import roc_auc_score

print("roc_auc test set", roc_auc_score(y_test, xgb_classifier.predict_proba(X_test)[:,1]))

print("roc_auc training set", roc_auc_score(y_train, xgb_classifier.predict_proba(X_train)[:,1]))


roc_auc test set 0.8746933446917583
roc_auc training set 0.956679219864915


In [188]:
X_final = pd.concat([X_train, X_test])
Y_final = pd.concat([y_train, y_test])

In [189]:
X_final.head()
print(len(X_final))
print(len(Y_final))

54808
54808


In [190]:
# train with the whole data set

xgb_classifier.fit(X_final,Y_final)

XGBClassifier(base_score=0.4, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.25, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [191]:
#Y_val = X_val['is_promoted']
X_val= X_val.drop(columns=['is_promoted'])

In [192]:
X_val.head()

,no_of_trainings,awards_won,avg_training_score,kpi_prev_rating,year_tobe_served,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,region_region_10,region_region_11,region_region_12,region_region_13,region_region_14,region_region_15,region_region_16,region_region_17,region_region_18,region_region_19,region_region_2,region_region_20,region_region_21,region_region_22,region_region_23,region_region_24,region_region_25,region_region_26,region_region_27,region_region_28,region_region_29,region_region_3,region_region_30,region_region_31,region_region_32,region_region_33,region_region_34,region_region_4,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,education_Below Secondary,education_Master's & above,gender_m,recruitment_channel_referred,recruitment_channel_sourcing
employee_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8724,1,0,77,4.267911,37,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
74430,1,0,51,3.000000,34,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
72255,1,0,47,1.000000,33,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
38562,3,0,65,2.000000,38,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
64486,1,0,61,4.000000,37,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [193]:
y_pred_val = xgb_classifier.predict(X_val)

In [194]:
y_pred_val

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [195]:
X_val['pred_is_promoted'] = y_pred_val

In [196]:
X_val.pred_is_promoted.sum()

760

In [197]:
submission_data = X_val.copy()
submission_data['is_promoted'] = y_pred_val

In [198]:
submission_data = submission_data[['is_promoted']]

In [199]:
submission_data.head()

,is_promoted
employee_id,
8724,0
74430,0
72255,0
38562,0
64486,0


In [200]:
submission_data.to_csv('F:\\career\\Udemy_projects\\HR analytics\\submission_kpi_prevrat_60-age+leng_wo_3cols_base04_9.csv')